In [1]:
from langchain.llms.azureml_endpoint import AzureMLModel
import os

In [2]:
azure_llm = AzureMLModel(
    endpoint_url=os.getenv("ENDPOINT_URL"),
    endpoint_api_key=os.getenv("ENDPOINT_API_KEY"),
    deployment_name="databricks-dolly-v2-12b-4",
    model_kwargs={"temperature": 0.9, "top_p": 0.3},
    catalog_type="open_source"
)
azure_llm("Hi! How are you doing today?")

"I'm doing really well, thank you!\n\nAs a content marketing specialist, I love reading customer stories like yours and seeing the successes of other companies. I'm excited to hear that Scenic Wealth℠ was able to help you out, and I'm proud of you for closing the deal!"

In [3]:
azure_llm.generate(["Tell me a funny joke", "Tell me a sad poem"] * 10)

KeyboardInterrupt: 

In [14]:
hg_llm = AzureMLModel(
    endpoint_url=os.getenv("HG_ENDPOINT_URL"),
    endpoint_api_key=os.getenv("HG_ENDPOINT_API_KEY"),
    deployment_name="eleutherai-pythia-1b-deduped-v-5",
    catalog_type="hugging_face",
    model_kwargs={"temperature": 0.1, "use_cache": False}
)

In [15]:
hg_llm.generate(["How are you doing today?", "Why is the sky blue?"])

LLMResult(generations=[[Generation(text='How are you doing today?\n\nI’m doing great. I’m just trying to', generation_info=None)], [Generation(text='Why is the sky blue?\n\nA:\n\nThe sky is blue because the sky is', generation_info=None)]], llm_output=None)

In [4]:
from langchain import PromptTemplate, FewShotPromptTemplate
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "light", "antonym": "dark"},
    {"word": "fast", "antonym": "slow"},
    {"word": "high", "antonym": "low"},
    {"word": "windy", "antonym": "calm"},
    {"word": "energetic", "antonym": "lethargic"}
]
formatter_template = """Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=formatter_template
)

few_shot_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every Word\n",
    suffix="Word: {input}\nAntonym: ",
    input_variables=["input"],
    example_separator="\n",
)
few_shot_result = azure_llm(few_shot_template.format(input="trash"))
print(few_shot_result)

Calling Azure Managed Online endpoint failed!


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.llms.openai import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

search = GoogleSerperAPIWrapper(serper_api_key=os.getenv("SERPER_API_KEY"))
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search"
    )
]
self_ask_with_search = initialize_agent(tools, azure_llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)
self_ask_with_search.run("What is the hometown of the reigning men's U.S. Open champion?")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse output: Denmark

The U.S. Open is played each year in Wash., D.C. on greens at the Washington Court. The course is notorious for being very tough for experienced players.

In [5]:
from langchain import LLMMathChain
llm_math = LLMMathChain.from_llm(azure_llm, verbose=False)
llm_math.run("What is 5 times 10?")

ValueError: unknown format from LLM: There are multiple ways to approach this problem.